In [1]:
#Basic model with random numbers

import numpy as np
import pandas as pd
from arch import arch_model
 
np.random.seed(0)
returns = np.random.normal(loc=0.001, scale=0.01, size=1000)
 
date_range = pd.date_range(start='2023-01-01', periods=len(returns), freq='B')
data = pd.DataFrame(data={'Returns': returns}, index=date_range)
 
p = 1
q = 1
model = arch_model(data['Returns'], vol='Garch', p=p, q=q)
results = model.fit()
print(results.summary())
forecast = results.forecast(horizon=1)
print("Forecasted volatility for the next day:", forecast.variance[-1:].values[0, 0])


Iteration:      1,   Func. Count:      6,   Neg. LLF: 78556308.41893135
Iteration:      2,   Func. Count:     18,   Neg. LLF: 412209050948429.25
Iteration:      3,   Func. Count:     30,   Neg. LLF: 38305178774632.72
Iteration:      4,   Func. Count:     42,   Neg. LLF: 9875938.416923266
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3199.5088791421113
            Iterations: 6
            Function evaluations: 50
            Gradient evaluations: 4
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                Returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                3199.51
Distribution:                  Normal   AIC:                          -6391.02
Method:            Maximum Likelihood   BIC:                          -6371.39
                            

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 9.742e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [2]:
import ccxt
import pandas as pd
import numpy as np
from arch import arch_model

# 1) Create an exchange object for Kraken
exchange = ccxt.kraken()

# 2) Set parameters
symbol = 'BTC/USD'       # BTC-USD pair on Kraken
timeframe = '1m'         # 1-minute data
limit = 720             # number of candles (~12 hours if 1 candle=1min)

# 3) Fetch OHLCV data
#   - Each entry is [timestamp_ms, open, high, low, close, volume]
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 4) Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# 5) Compute returns
df['Returns'] = df['close'].pct_change()

# 6) Replace infinities with NaN, drop missing
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=['Returns'], inplace=True)

# 7) Fit a GARCH(1,1) model
p, q = 1, 1
model = arch_model(df['Returns'], vol='GARCH', p=p, q=q)
results = model.fit(disp='off')
print(results.summary())

# 8) Forecast next 60 steps (1-minute each)
forecast_horizon = 10
forecast = results.forecast(horizon=forecast_horizon)
predicted_variance = forecast.variance.iloc[-1]
predicted_volatility = np.sqrt(predicted_variance.values)

print("\nForecasted volatility for each of the next 60 minutes:")
print(predicted_volatility)

                     Constant Mean - GARCH Model Results                      
Dep. Variable:                Returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                3757.63
Distribution:                  Normal   AIC:                          -7507.27
Method:            Maximum Likelihood   BIC:                          -7488.96
                                        No. Observations:                  719
Date:                Tue, Feb 25 2025   Df Residuals:                      718
Time:                        13:55:04   Df Model:                            1
                                   Mean Model                                  
                  coef    std err          t      P>|t|        95.0% Conf. Int.
-------------------------------------------------------------------------------
mu         -6.4119e-05  2.154e-06    -29.761 1.26

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.191e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


In [4]:
#EGARCH MODEL

import ccxt
import pandas as pd
import numpy as np
from arch import arch_model

# 1) Create an exchange object for Kraken
exchange = ccxt.kraken()

# 2) Set parameters
symbol = 'BTC/USD'       # BTC-USD pair on Kraken
timeframe = '1m'         # 1-minute data
limit = 720             # number of candles (~12 hours if 1 candle=1min)

# 3) Fetch OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 4) Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# 5) Compute returns
df['Returns'] = df['close'].pct_change()

# 6) Replace infinities with NaN, drop missing
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=['Returns'], inplace=True)

# 7) Fit an EGARCH(1,1) model
p, q = 1, 1
model = arch_model(df['Returns'], vol='EGARCH', p=p, q=q)
results = model.fit(disp='off')
print(results.summary())

# 8) Forecast next 10 steps (1-minute each)
forecast_horizon = 10
forecast = results.forecast(horizon=forecast_horizon, method="simulation", simulations=1000)
predicted_variance = forecast.variance.iloc[-1]
predicted_volatility = np.sqrt(predicted_variance.values)

print("\nForecasted volatility for each of the next 10 minutes:")
print(predicted_volatility)


                     Constant Mean - EGARCH Model Results                     
Dep. Variable:                Returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                     EGARCH   Log-Likelihood:           -2.77973e+06
Distribution:                  Normal   AIC:                       5.55947e+06
Method:            Maximum Likelihood   BIC:                       5.55949e+06
                                        No. Observations:                  719
Date:                Tue, Feb 25 2025   Df Residuals:                      718
Time:                        13:56:50   Df Model:                            1
                                   Mean Model                                  
                  coef    std err          t      P>|t|        95.0% Conf. Int.
-------------------------------------------------------------------------------
mu         -6.3744e-05  1.930e-06    -33.028 3.24

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.19e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


In [ ]:
import ccxt
import pandas as pd
import numpy as np
from arch import arch_model

# 1) Create an exchange object for Kraken
exchange = ccxt.kraken()

# 2) Set parameters
symbol = 'BTC/USD'       # BTC-USD pair on Kraken
timeframe = '1m'         # 1-minute data
limit = 720             # number of candles (~12 hours if 1 candle=1min)

# 3) Fetch OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 4) Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# 5) Compute returns
df['Returns'] = df['close'].pct_change()

# 6) Replace infinities with NaN, drop missing
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=['Returns'], inplace=True)

# 7) Fit an EGARCH(1,1) model
p, q = 1, 1
model_egarch = arch_model(df['Returns'], vol='EGARCH', p=p, q=q)
results_egarch = model_egarch.fit(disp='off')
print(results_egarch.summary())

# 8) Forecast next 10 steps (1-minute each) using Monte Carlo simulation
forecast_horizon = 10
forecast_egarch = results_egarch.forecast(horizon=forecast_horizon, method="simulation", simulations=1000)
predicted_variance_egarch = forecast_egarch.variance.iloc[-1]
predicted_volatility_egarch = np.sqrt(predicted_variance_egarch.values)

print("\nForecasted volatility for each of the next 10 minutes (EGARCH):")
print(predicted_volatility_egarch)

# 9) Fit a GJR-GARCH(1,1) model
model_gjr = arch_model(df['Returns'], vol='GARCH', p=p, o=1, q=q)
results_gjr = model_gjr.fit(disp='off')
print(results_gjr.summary())

# 10) Forecast next 10 steps (1-minute each) using Monte Carlo simulation for GJR-GARCH
forecast_gjr = results_gjr.forecast(horizon=forecast_horizon, method="simulation", simulations=1000)
predicted_variance_gjr = forecast_gjr.variance.iloc[-1]
predicted_volatility_gjr = np.sqrt(predicted_variance_gjr.values)

print("\nForecasted volatility for each of the next 10 minutes (GJR-GARCH):")
print(predicted_volatility_gjr)